# Intro to Recommender Systems Lab

Complete the exercises below to solidify your knowledge and understanding of recommender systems.

For this lab, we are going to be putting together a user similarity based recommender system in a step-by-step fashion. Our data set contains customer grocery purchases, and we will use similar purchase behavior to inform our recommender system. Our recommender system will generate 5 recommendations for each customer based on the purchases they have made.

In [1]:
import pandas as pd
from scipy.spatial.distance import pdist, squareform
import numpy as np

In [2]:
data = pd.read_csv('../data/customer_product_sales.csv')

In [3]:
data

,CustomerID,FirstName,LastName,SalesID,ProductID,ProductName,Quantity
0,61288,Rosa,Andersen,134196,229,Bread - Hot Dog Buns,16
1,77352,Myron,Murray,6167892,229,Bread - Hot Dog Buns,20
2,40094,Susan,Stevenson,5970885,229,Bread - Hot Dog Buns,11
3,23548,Tricia,Vincent,6426954,229,Bread - Hot Dog Buns,6
4,78981,Scott,Burch,819094,229,Bread - Hot Dog Buns,20
...,...,...,...,...,...,...,...
68579,94547,Gerald,Donovan,687884,211,Wine - Prosecco Valdobiaddene,24
68580,57654,Lashonda,Roy,4373415,211,Wine - Prosecco Valdobiaddene,15
68581,36751,Glen,Frederick,2405597,211,Wine - Prosecco Valdobiaddene,10
68582,43270,Devon,Mc Daniel,4328852,211,Wine - Prosecco Valdobiaddene,11


## Step 1: Create a data frame that contains the total quantity of each product purchased by each customer.

You will need to group by CustomerID and ProductName and then sum the Quantity field.

In [4]:
groupbydata = data.groupby(by  = ['CustomerID', 'ProductName'], dropna = False).sum('Quantity').sort_values(by = 'Quantity', ascending = False)
groupbydata

SalesID  ProductID  Quantity
CustomerID ProductName                                                     
90069      Longos - Grilled Salmon With Bbq   18271598        784        92
80694      Yeast Dry - Fermipan                7654976        124        84
96615      Veal - Eye Of Round                15919656        777        75
97063      Bread - French Baquette             9104563        321        75
95017      Coffee - Irish Cream                9358869        384        75
...                                                ...        ...       ...
2617       Pasta - Cheese / Spinach Bauletti   1860958        348         1
           Pail For Lid 1537                   5243961        347         1
           Oil - Shortening,liqud, Fry         1565107        335         1
           Muffin Chocolate Individual Wrap    3227306        431         1
33         Apricots - Dried                     172592        324         1

[63628 rows x 3 columns]

## Step 2: Use the `pivot_table` method to create a product by customer matrix.

The rows of the matrix should represent the products, the columns should represent the customers, and the values should be the quantities of each product purchased by each customer. You will also need to replace nulls with zeros, which you can do using the `fillna` method.

In [5]:
table = pd.pivot_table(groupbydata, values='Quantity', index=['ProductName'],
                       columns=['CustomerID'], aggfunc=np.sum, fill_value=0)

# aggfunc=lambda x: len(x.unique())
# aggfunc=pd.Series.nunique
'''
>>> table
C        large  small
A   B
bar one      4      5
    two      7      6
foo one      4      1
    two      0      6
    
    
     A    B      C  D  E
0  foo  one  small  1  2
1  foo  one  large  2  4
2  foo  one  large  2  5
3  foo  two  small  3  5
4  foo  two  small  3  6
5  bar  one  large  4  6
6  bar  one  small  5  8
7  bar  two  small  6  9
8  bar  two  large  7  9
'''
table

CustomerID,33,200,264,356,412,464,477,639,649,669,...,97697,97753,97769,97793,97900,97928,98069,98159,98185,98200
ProductName,,,,,,,,,,,,,,,,,,,,,
Anchovy Paste - 56 G Tube,0,0,0,0,0,0,0,1,0,0,...,0,25,0,0,0,0,0,0,0,0
"Appetizer - Mini Egg Roll, Shrimp",0,0,0,0,0,0,0,0,0,0,...,25,25,0,0,0,0,0,0,0,0
Appetizer - Mushroom Tart,0,0,0,0,0,0,0,1,0,0,...,25,0,0,0,0,0,0,0,25,0
Appetizer - Sausage Rolls,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,25,25,25,0,25,0
Apricots - Dried,1,0,0,0,1,0,0,0,0,0,...,0,25,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
Yeast Dry - Fermipan,0,0,0,0,0,0,0,0,0,0,...,0,0,0,25,0,0,0,0,0,0
Yoghurt Tubes,0,0,0,0,0,0,0,0,2,0,...,0,0,0,0,0,0,0,25,0,0
"Yogurt - Blueberry, 175 Gr",0,1,0,0,0,0,0,0,0,0,...,25,0,0,25,0,0,0,0,0,0


In [6]:
tableT = table.T
tableT

ProductName,Anchovy Paste - 56 G Tube,"Appetizer - Mini Egg Roll, Shrimp",Appetizer - Mushroom Tart,Appetizer - Sausage Rolls,Apricots - Dried,Apricots - Halves,Apricots Fresh,Arizona - Green Tea,Artichokes - Jerusalem,Assorted Desserts,...,"Wine - White, Colubia Cresh","Wine - White, Mosel Gold","Wine - White, Schroder And Schyl",Wine - Wyndham Estate Bin 777,Wonton Wrappers,Yeast Dry - Fermipan,Yoghurt Tubes,"Yogurt - Blueberry, 175 Gr",Yogurt - French Vanilla,Zucchini - Yellow
CustomerID,,,,,,,,,,,,,,,,,,,,,
33,0,0,0,0,1,0,0,0,0,1,...,0,0,0,0,0,0,0,0,1,0
200,0,0,0,0,0,0,0,0,0,0,...,1,0,0,0,0,0,0,1,0,0
264,0,0,0,0,0,1,1,0,0,0,...,0,0,0,1,0,0,0,0,0,0
356,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
412,0,0,0,0,1,0,0,0,0,0,...,0,1,1,1,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
97928,0,0,0,25,0,50,0,25,0,0,...,0,25,25,0,0,0,0,0,0,0
98069,0,0,0,25,0,25,0,0,0,25,...,0,0,0,0,0,0,0,0,0,0
98159,0,0,0,0,0,0,0,0,0,0,...,0,50,0,0,0,0,25,0,0,0


## Step 3: Create a customer similarity matrix using `squareform` and `pdist`. For the distance metric, choose "euclidean."

In [7]:
data_norm = pd.DataFrame(1/(1 + squareform(pdist(tableT, 'euclidean'))),
                                index=table.columns,
                                columns=table.columns)

data_norm

CustomerID,33,200,264,356,412,464,477,639,649,669,...,97697,97753,97769,97793,97900,97928,98069,98159,98185,98200
CustomerID,,,,,,,,,,,,,,,,,,,,,
33,1.000000,0.077421,0.087047,0.081800,0.080634,0.082709,0.074573,0.083020,0.081503,0.080070,...,0.004811,0.004669,0.004412,0.005019,0.004312,0.004515,0.004583,0.004355,0.004167,0.004333
200,0.077421,1.000000,0.078448,0.076435,0.073693,0.075255,0.075956,0.076435,0.077674,0.076923,...,0.004824,0.004681,0.004431,0.005047,0.004311,0.004521,0.004614,0.004367,0.004166,0.004335
264,0.087047,0.078448,1.000000,0.080070,0.081800,0.080350,0.076923,0.080634,0.082100,0.078448,...,0.004822,0.004674,0.004416,0.005035,0.004322,0.004543,0.004595,0.004365,0.004179,0.004333
356,0.081800,0.076435,0.080070,1.000000,0.076435,0.078187,0.075025,0.082403,0.077171,0.075956,...,0.004816,0.004671,0.004416,0.005038,0.004310,0.004526,0.004578,0.004365,0.004175,0.004339
412,0.080634,0.073693,0.081800,0.076435,1.000000,0.078711,0.075025,0.082403,0.078187,0.078448,...,0.004810,0.004702,0.004414,0.005034,0.004318,0.004530,0.004578,0.004367,0.004177,0.004349
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
97928,0.004515,0.004521,0.004543,0.004526,0.004530,0.004533,0.004534,0.004526,0.004528,0.004523,...,0.003509,0.003509,0.003300,0.003653,0.003579,1.000000,0.003638,0.003418,0.003255,0.003245
98069,0.004583,0.004614,0.004595,0.004578,0.004578,0.004578,0.004589,0.004581,0.004585,0.004592,...,0.003509,0.003509,0.003369,0.003509,0.003496,0.003638,1.000000,0.003469,0.003345,0.003381
98159,0.004355,0.004367,0.004365,0.004365,0.004367,0.004363,0.004366,0.004363,0.004369,0.004365,...,0.003509,0.003565,0.003322,0.003431,0.003322,0.003418,0.003469,1.000000,0.003277,0.003266


## Step 4: Check your results by generating a list of the top 5 most similar customers for a specific CustomerID.

In [8]:
data_norm[264]

CustomerID
33       0.087047
200      0.078448
264      1.000000
356      0.080070
412      0.081800
           ...   
97928    0.004543
98069    0.004595
98159    0.004365
98185    0.004179
98200    0.004333
Name: 264, Length: 1000, dtype: float64

In [9]:
data_simil_norm = data_norm[264].sort_values(ascending=False)[1:]
data_simil_norm = data_simil_norm[:5]
data_simil_norm

CustomerID
1008    0.088152
2617    0.088152
1072    0.087410
33      0.087047
3535    0.086333
Name: 264, dtype: float64

## Step 5: From the data frame you created in Step 1, select the records for the list of similar CustomerIDs you obtained in Step 4.

In [10]:
similar_customers_series = pd.DataFrame(data_simil_norm)

selected_records = groupbydata.loc[similar_customers_series.index]

selected_records


SalesID  ProductID  Quantity
CustomerID ProductName                                                  
1008       Cheese - Bocconcini              3731139        192         2
           Corn Meal                        2865936        275         1
           Island Oasis - Mango Daiquiri     596538        127         1
           Gloves - Goldtouch Disposable    3066548        171         1
           Gatorade - Xfactor Berry         2035963        448         1
...                                             ...        ...       ...
3535       Ocean Spray - Ruby Red            622468        115         1
           Ocean Spray - Kiwi Strawberry    5454259        113         1
           Muffin Batt - Blueberry Passion  5375388        121         1
           Milk Powder                       813539        353         1
           Milk - 2%                        6648211        168         1

[288 rows x 3 columns]

## Step 6: Aggregate those customer purchase records by ProductName, sum the Quantity field, and then rank them in descending order by quantity.

This will give you the total number of each product purchased by the 5 most similar customers to the customer you selected in order from most purchased to least.

In [11]:
grouped_purchases = selected_records.groupby('ProductName')['Quantity'].sum().reset_index()
sorted_purchases = grouped_purchases.sort_values(by = 'Quantity', ascending = False).head()
sorted_purchases

,ProductName,Quantity
177,Soupfoamcont12oz 112con,3
218,Wine - Two Oceans Cabernet,3
36,Bread - Italian Roll With Herbs,3
199,"Veal - Inside, Choice",3
148,Potatoes - Idaho 100 Count,3


In [12]:
list_ProductName = list(sorted_purchases['ProductName'])

## Step 7: Filter the list for products that the chosen customer has not yet purchased and then recommend the top 5 products with the highest quantities that are left.

- Merge the ranked products data frame with the customer product matrix on the ProductName field.
- Filter for records where the chosen customer has not purchased the product.
- Show the top 5 results.

In [13]:
customer264 = data[data['CustomerID'].isin([264])]#.sort_values(by='price', ascending=False)
customer264

,CustomerID,FirstName,LastName,SalesID,ProductID,ProductName,Quantity
159,264,Virgil,Small,24688,270,Cheese Cloth No 100,1
3117,264,Virgil,Small,2229328,85,Cheese - Parmesan Cubes,1
7544,264,Virgil,Small,4491863,168,Milk - 2%,1
8578,264,Virgil,Small,6164483,199,Pork - Hock And Feet Attached,1
9306,264,Virgil,Small,1848293,371,Creme De Banane - Marie,1
...,...,...,...,...,...,...,...
62804,264,Virgil,Small,3331643,319,Towels - Paper / Kraft,1
64489,264,Virgil,Small,1537297,398,Wine - Wyndham Estate Bin 777,1
65941,264,Virgil,Small,448565,425,Wine - Redchard Merritt,1
66436,264,Virgil,Small,3526383,378,Bread Foccacia Whole,1


In [14]:
data_merge = pd.merge(grouped_purchases, table, on = 'ProductName' )
data_merge = data_merge[data_merge[264].isin([0])].sort_values(by = 'Quantity', ascending = False).head()
data_merge[['ProductName', 'Quantity']]

,ProductName,Quantity
177,Soupfoamcont12oz 112con,3
218,Wine - Two Oceans Cabernet,3
199,"Veal - Inside, Choice",3
36,Bread - Italian Roll With Herbs,3
84,Fondant - Icing,3


## Step 8: Now that we have generated product recommendations for a single user, put the pieces together and iterate over a list of all CustomerIDs.

- Create an empty dictionary that will hold the recommendations for all customers.
- Create a list of unique CustomerIDs to iterate over.
- Iterate over the customer list performing steps 4 through 7 for each and appending the results of each iteration to the dictionary you created.

In [15]:
dict_ = {}
for i in data_norm.columns:
    data_simil_norm = data_norm[i].sort_values(ascending=False)[1:]
    data_simil_norm = data_simil_norm[:5]
    similar_customers_series = pd.DataFrame(data_simil_norm)
    selected_records = groupbydata.loc[similar_customers_series.index]
    grouped_purchases = selected_records.groupby('ProductName')['Quantity'].sum().reset_index()
    sorted_purchases = grouped_purchases.sort_values(by = 'Quantity', ascending = False).head()
    list_ProductName = list(sorted_purchases['ProductName'])
    customerID_ = data[data['CustomerID'].isin([i])]#.sort_values(by='price', ascending=False)
    data_merge = pd.merge(grouped_purchases, table, on = 'ProductName' )
    data_merge = data_merge[data_merge[i].isin([0])].sort_values(by = 'Quantity', ascending = False).head()
    data_merge[['ProductName', 'Quantity']]
    dict_[i] = (list_ProductName)
    


##  Step 9: Store the results in a Pandas data frame. The data frame should a column for Customer ID and then a column for each of the 5 product recommendations for each customer.

In [16]:
all_customers_recomendations = pd.DataFrame(dict_).T

In [17]:
all_customers_recomendations["Recomendations"] = all_customers_recomendations[[0, 1, 2, 3, 4]].apply("-".join, axis=1)
all_customers_recomendations = all_customers_recomendations["Recomendations"]
pd.DataFrame(all_customers_recomendations)

,Recomendations
33,Butter - Unsalted-Wine - Ej Gallo Sierra Valle...
200,Soup - Campbells Bean Medley-Muffin - Carrot I...
264,Soupfoamcont12oz 112con-Wine - Two Oceans Cabe...
356,"Butter - Unsalted-Veal - Inside, Choice-Wine -..."
412,Olive - Spread Tapenade-Sprouts - Baby Pea Ten...
...,...
97928,Bouq All Italian - Primerba-Tea - Jasmin Green...
98069,Skirt - 29 Foot-Chocolate - Dark-Beans - Kidne...
98159,Squid - Tubes / Tenticles 10/20-Chips Potato A...
98185,Halibut - Steaks-Cod - Black Whole Fillet-Wine...


## Step 10: Change the distance metric used in Step 3 to something other than euclidean (correlation, cityblock, consine, jaccard, etc.). Regenerate the recommendations for all customers and note the differences.

In [18]:
data_norm_other = pd.DataFrame(1/(1 + squareform(pdist(tableT, 'cosine'))),
                                index=table.columns,
                                columns=table.columns)

data_norm_other

CustomerID,33,200,264,356,412,464,477,639,649,669,...,97697,97753,97769,97793,97900,97928,98069,98159,98185,98200
CustomerID,,,,,,,,,,,,,,,,,,,,,
33,1.000000,0.530003,0.563492,0.543546,0.542393,0.541054,0.522016,0.530783,0.533096,0.535293,...,0.530306,0.517987,0.527662,0.511471,0.534181,0.513809,0.528795,0.520208,0.529481,0.527142
200,0.530003,1.000000,0.537166,0.535098,0.521226,0.519193,0.556945,0.516729,0.536350,0.541451,...,0.545224,0.533067,0.555246,0.543554,0.530308,0.521925,0.573177,0.537284,0.526167,0.527280
264,0.563492,0.537166,1.000000,0.531963,0.550168,0.525368,0.538967,0.514932,0.537046,0.524193,...,0.546880,0.525550,0.535062,0.531806,0.553085,0.560225,0.548538,0.538340,0.554929,0.527142
356,0.543546,0.535098,0.531963,1.000000,0.525982,0.523986,0.536792,0.540890,0.516901,0.519482,...,0.536321,0.520565,0.533119,0.534066,0.528874,0.530406,0.520138,0.536202,0.544727,0.535974
412,0.542393,0.521226,0.550168,0.525982,1.000000,0.534073,0.542308,0.547743,0.530459,0.543050,...,0.527903,0.565369,0.528847,0.529278,0.541674,0.536686,0.519628,0.538710,0.546943,0.552636
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
97928,0.513809,0.521925,0.560225,0.530406,0.536686,0.542873,0.540381,0.533190,0.535224,0.526495,...,0.532674,0.539309,0.522883,0.542570,0.569539,1.000000,0.562050,0.543657,0.531363,0.519127
98069,0.528795,0.573177,0.548538,0.520138,0.519628,0.520835,0.534221,0.525789,0.531918,0.541402,...,0.529233,0.535979,0.530224,0.518742,0.554911,0.562050,1.000000,0.548011,0.542143,0.536728
98159,0.520208,0.537284,0.538340,0.536202,0.538710,0.533856,0.535601,0.535704,0.544926,0.535721,...,0.542777,0.556893,0.535487,0.521507,0.541404,0.543657,0.548011,1.000000,0.543258,0.531447


In [19]:
dict_other = {}
for i in data_norm_other.columns:
    data_simil_norm_other = data_norm_other[i].sort_values(ascending=False)[1:]
    data_simil_norm_other = data_simil_norm_other[:5]
    similar_customers_series_other = pd.DataFrame(data_simil_norm_other)
    selected_records_other = groupbydata.loc[similar_customers_series_other.index]
    grouped_purchases_other = selected_records_other.groupby('ProductName')['Quantity'].sum().reset_index()
    sorted_purchases_other = grouped_purchases_other.sort_values(by = 'Quantity', ascending = False).head()
    list_ProductName_other = list(sorted_purchases_other['ProductName'])
    customerID__other = data[data['CustomerID'].isin([i])]#.sort_values(by='price', ascending=False)
    data_merge_other = pd.merge(grouped_purchases_other, table, on = 'ProductName' )
    data_merge_other = data_merge_other[data_merge_other[i].isin([0])].sort_values(by = 'Quantity', ascending = False).head()
    data_merge_other[['ProductName', 'Quantity']]
    dict_other[i] = (list_ProductName_other)
    


In [20]:
dict_other

{33: ['Knife Plastic - White',
  'Phyllo Dough',
  'Cake - Box Window 10x10x2.5',
  'Soup - Campbells, Beef Barley',
  'Onions - Cippolini'],
 200: ['Olives - Kalamata',
  'Mustard Prepared',
  'Quiche Assorted',
  'Pop Shoppe Cream Soda',
  'Longos - Grilled Salmon With Bbq'],
 264: ['Lemonade - Natural, 591 Ml',
  'Pickerel - Fillets',
  'Muffin Mix - Blueberry',
  'Wine - Blue Nun Qualitatswein',
  'Water - Mineral, Natural'],
 356: ['Tomatoes Tear Drop',
  'Sobe - Tropical Energy',
  'Bread - English Muffin',
  'Thyme - Lemon, Fresh',
  'Olive - Spread Tapenade'],
 412: ['Banana - Leaves',
  'Salsify, Organic',
  'Beans - Wax',
  'Durian Fruit',
  'Whmis - Spray Bottle Trigger'],
 464: ['Lambcasing',
  'Carbonated Water - Cherry',
  'Wonton Wrappers',
  'Pail With Metal Handle 16l White',
  'Bread - Bistro White'],
 477: ['Bacardi Breezer - Tropical',
  'Halibut - Fletches',
  'Chocolate - Feathers',
  'Wine - Prosecco Valdobiaddene',
  'Pasta - Penne, Rigate, Dry'],
 639: ['Juice 

In [21]:
all_customers_recomendations_other = pd.DataFrame(dict_other).T

In [22]:
all_customers_recomendations_other["Recomendations_other"] = all_customers_recomendations_other[[0, 1, 2, 3, 4]].apply("-".join, axis=1)
all_customers_recomendations_other = all_customers_recomendations_other["Recomendations_other"]
all_customers_recomendations_other = pd.DataFrame(all_customers_recomendations_other)
all_customers_recomendations_other

,Recomendations_other
33,Knife Plastic - White-Phyllo Dough-Cake - Box ...
200,Olives - Kalamata-Mustard Prepared-Quiche Asso...
264,"Lemonade - Natural, 591 Ml-Pickerel - Fillets-..."
356,Tomatoes Tear Drop-Sobe - Tropical Energy-Brea...
412,"Banana - Leaves-Salsify, Organic-Beans - Wax-D..."
...,...
97928,Apricots - Halves-Cheese - Mozzarella-Tofu - F...
98069,Mustard Prepared-Longos - Grilled Salmon With ...
98159,Squid - Tubes / Tenticles 10/20-Crush - Cream ...
98185,"Peas - Pigeon, Dry-Halibut - Fletches-Chicken ..."


In [23]:
all_customers_recomendations_other.join(all_customers_recomendations)

,Recomendations_other,Recomendations
33,Knife Plastic - White-Phyllo Dough-Cake - Box ...,Butter - Unsalted-Wine - Ej Gallo Sierra Valle...
200,Olives - Kalamata-Mustard Prepared-Quiche Asso...,Soup - Campbells Bean Medley-Muffin - Carrot I...
264,"Lemonade - Natural, 591 Ml-Pickerel - Fillets-...",Soupfoamcont12oz 112con-Wine - Two Oceans Cabe...
356,Tomatoes Tear Drop-Sobe - Tropical Energy-Brea...,"Butter - Unsalted-Veal - Inside, Choice-Wine -..."
412,"Banana - Leaves-Salsify, Organic-Beans - Wax-D...",Olive - Spread Tapenade-Sprouts - Baby Pea Ten...
...,...,...
97928,Apricots - Halves-Cheese - Mozzarella-Tofu - F...,Bouq All Italian - Primerba-Tea - Jasmin Green...
98069,Mustard Prepared-Longos - Grilled Salmon With ...,Skirt - 29 Foot-Chocolate - Dark-Beans - Kidne...
98159,Squid - Tubes / Tenticles 10/20-Crush - Cream ...,Squid - Tubes / Tenticles 10/20-Chips Potato A...
98185,"Peas - Pigeon, Dry-Halibut - Fletches-Chicken ...",Halibut - Steaks-Cod - Black Whole Fillet-Wine...
